In [65]:
import sys
sys.path.append('../keras_network/')
from keras.callbacks import TensorBoard, LearningRateScheduler
import tensorflow as tf
from configargparse import ArgParser
from models import conv_model, fc_model, conv_model_tile
from losses import *
from utils import *
from data_generator import DataGenerator
import matplotlib.pyplot
%matplotlib inline
import keras.backend as K
from keras.layers import *
from keras.models import Model

In [3]:
# Reproduce the variance issue

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

In [5]:
data_dir = '/scratch/srasp/preprocessed_data/'
train_fn = '/purecrm_ess_train_sample1_shuffle'

In [6]:
train_gen = DataGenerator(
        data_dir,
        train_fn + '_features.nc',
        train_fn + '_targets.nc',
        1024,
        shuffle=True,
    )

Generator will have 23494656 samples in 22944 batches
Features have shape 152; targets have shape 120


In [7]:
feature_shape = train_gen.feature_shape
target_shape = train_gen.target_shape

In [39]:
def var_true(y_true, y_pred): return K.mean(K.var(y_true, axis=(0,1)))
def var_pred(y_true, y_pred): return K.mean(K.var(y_pred, axis=(0,1)))

In [40]:
def var_ratio(y_true, y_pred):
    vt = var_true(y_true, y_pred)
    vp = var_pred(y_true, y_pred)
    return vp / vt

In [58]:
def var_loss(y_true, y_pred):
    vt = var_true(y_true, y_pred)
    vp = var_pred(y_true, y_pred)
    return K.square(vp - vt)

In [60]:
model = fc_model(
    feature_shape,
    target_shape,
    [512],
    1e-3,
    'mse',
    batch_norm=False,
    activation='tanh'
)
model.compile(keras.optimizers.Adam(1e-3), loss='mse', 
              metrics=[var_true, var_pred, var_ratio, var_loss])

In [61]:
model.fit_generator(
        train_gen.return_generator(),
        train_gen.n_batches,
        epochs=1,
        workers=8,
        max_queue_size=50,
    )

Epoch 1/1
 6592/22944 [=======>......................] - ETA: 6:38 - loss: 0.0034 - var_true: 0.0074 - var_pred: 0.0055 - var_ratio: 0.7377 - var_loss: 6.4968e-

KeyboardInterrupt: 

In [ ]:
from keras.losses import mse

In [ ]:
def approx_crps_cat(bin_width):
    """Wrapper to pass bin_width as an argument to the loss function.
    Args:
        bin_width: width of categorical bins
    Returns:
        loss_function: approximate crps_loss function with bin_width specified
    """
    def loss(y_true, y_pred):
        """Approximate CRPS function for categorical output.
        Args:
            y_true: One-hot-encoded output
            y_pred: Probability for each bin
        Returns:
            approx_crps: Approximate mean CRPS value for batch
        """
        # [sample, cat]
        cum_obs = K.cumsum(y_true, axis=1)
        cum_preds = K.cumsum(y_pred, axis=1)
        approx_crps = K.sum(K.square(cum_obs - cum_preds), axis=1) * bin_width
        return K.mean(approx_crps)
    return loss

In [83]:
def mse_var(ratio):
    """
    specify ratio to multiply var loss with
    """
    def loss(y_true, y_pred):
        return K.mean(mse(y_true, y_pred)) + ratio * var_loss(y_true, y_pred)
    return loss

In [88]:
model = fc_model(
    feature_shape,
    target_shape,
    [512],
    1e-3,
    'mse',
    batch_norm=False,
    activation='tanh'
)
model.compile(keras.optimizers.Adam(1e-3), loss=mse_var(100), 
              metrics=[var_true, var_pred, var_ratio, var_loss, 'mse'])

In [89]:
model.fit_generator(
        train_gen.return_generator(),
        train_gen.n_batches,
        epochs=1,
        workers=8,
        max_queue_size=50,
    )

Epoch 1/1
  956/22944 [>.............................] - ETA: 15:42 - loss: 0.0611 - var_true: 0.0074 - var_pred: 0.0139 - var_ratio: 1.8696 - var_loss: 4.5323e-04 - mean_squared_error: 0.0158 ETA: 15:43 - loss: 0.0681 - var_true: 0.0074 - var_pred: 0.0146 - var_ratio: 1.9728 - - ETA: 15:34 - loss: 0.0646 - var_true: 0.0074 - var_pred: 0.0142

KeyboardInterrupt: 